# Gather Data from Oyez.org
This script will fetch the case data from oyez.org using async get requests to speed up the process. There are 8209 cases at the moment and roughly takes 1 hour to finish running.

Here's an already generated dataset:

Minimal JSON compact form (216MB):
https://www.dropbox.com/s/9kyk0dr2gf3ls23/oyez.json?dl=0

Prettified JSON human-readable form (431 MB):
https://www.dropbox.com/s/52a58aac8iujupv/oyez_pretty.json?dl=0

## Imports

In [1]:
# !pip install requests_futures
# !pip install json

from requests_futures.sessions import FuturesSession
import concurrent.futures as cf
import json

## Create an async session for get requests

In [2]:
session = FuturesSession()

In [3]:
x = [session.get('https://api.oyez.org/cases?per_page=1')]
for f in cf.as_completed(x):
    print(f.result().url)
    print(f.result().json())

https://api.oyez.org/cases?per_page=1
[{'ID': 49051, 'name': 'American Trucking Assns., Inc. v. United States', 'href': 'https://api.oyez.org/cases/1966/510', 'view_count': 0, 'docket_number': '510', 'timeline': [{'event': 'Decided', 'dates': [-124826400], 'href': 'https://api.oyez.org/case_timeline/case_timeline/51665'}], 'question': None, 'citation': {'volume': '382', 'page': '372', 'year': '1966', 'href': 'https://api.oyez.org/case_citation/case_citation/12979'}, 'term': '1966', 'description': None, 'justia_url': 'https://supreme.justia.com/cases/federal/us/382/372/'}]


## Fetch case data from oyez.org and convert it into JSON
Note that this data will not include the entire case data, but instead will have 'href' fields that points to the data.

In [4]:
future_data = session.get('https://api.oyez.org/cases?per_page=0')
response_data = future_data.result().json()

## Replace the 'href' fields in each case with the actual fetched data
This step takes the longest, roughly 45-60 minutes.

In [5]:
# Dictionary of the case data, key is case ID, value is the entire case JSON
data = {}

# A list of futures of all the async get requests
futures_cases = []
for i, element in enumerate(response_data):
    # response_data contains brief info about the case, instead we will
    # fetch each case individually from its 'href' field for more details
    href = element['href']
    futures_cases.append(session.get(href))

# Wait until all get requests are completed
# Order of completetion is arbitrary, first-done first-serve (the get requests)
futures_decisions = []
futures_opinions = []
futures_decisions_lookup = {}
futures_opinions_lookup = {}
for f in cf.as_completed(futures_cases):
    try:
        case_resp = f.result().json()
        
        # extract the case ID and save it to the data dictionary
        case_id = case_resp['ID']
        data[case_id] = case_resp
        
        # extract the decisions, if none exist, give it an empty list
        try:
            decisions = case_resp['decisions']
        except:
            decisions = []
        
        # extract the written opinions, give it an empty list
        try:
            opinions = case_resp['written_opinion']
        except:
            opinions = []
            
        # reset the decisions of the case, since we want to fill
        # it ourselves using the 'href' of each decision for more details
        case_resp['decisions'] = []
        for decision in decisions:
            try:
                # extract the decision href
                href = decision['href']
                futures_decisions.append(session.get(href))
                futures_decisions_lookup[href] = case_id
            except:
                pass
        
        # reset the opinions of the case, since we want to fill
        # it ourselves using the 'href' of each decision for more details
        case_resp['opinions'] = []
        for opinion in opinions:
            try:
                # extract the opinion href
                href = opinion['href']
                futures_opinions.append(session.get(href))
                futures_opinions_lookup[href] = case_id
            except:
                pass
        
        print(f'Case: {f.result().url}')
    except:
        pass

Case: https://api.oyez.org/cases/1967/1269
Case: https://api.oyez.org/cases/1971/70-5004
Case: https://api.oyez.org/cases/1971/71-6425
Case: https://api.oyez.org/cases/1971/71-452
Case: https://api.oyez.org/cases/1968/1047
Case: https://api.oyez.org/cases/1969/343
Case: https://api.oyez.org/cases/1966/1043
Case: https://api.oyez.org/cases/1971/70-161
Case: https://api.oyez.org/cases/1967/373
Case: https://api.oyez.org/cases/1971/70-5308
Case: https://api.oyez.org/cases/1971/70-5058
Case: https://api.oyez.org/cases/1966/510
Case: https://api.oyez.org/cases/1971/71-5144
Case: https://api.oyez.org/cases/1971/70-18
Case: https://api.oyez.org/cases/1971/69-4
Case: https://api.oyez.org/cases/1971/70-5039
Case: https://api.oyez.org/cases/1971/45_orig
Case: https://api.oyez.org/cases/1971/68-5009
Case: https://api.oyez.org/cases/1971/70-5014
Case: https://api.oyez.org/cases/1971/70-33
Case: https://api.oyez.org/cases/1971/70-79
Case: https://api.oyez.org/cases/1971/70-250
Case: https://api.oye

Case: https://api.oyez.org/cases/1971/70-5122
Case: https://api.oyez.org/cases/1972/72-6198
Case: https://api.oyez.org/cases/1971/70-113
Case: https://api.oyez.org/cases/1972/27_orig
Case: https://api.oyez.org/cases/1972/72-486
Case: https://api.oyez.org/cases/1972/71-873
Case: https://api.oyez.org/cases/1972/72-549
Case: https://api.oyez.org/cases/1972/72-438
Case: https://api.oyez.org/cases/1972/71-732
Case: https://api.oyez.org/cases/1972/72-5323
Case: https://api.oyez.org/cases/1972/71-507
Case: https://api.oyez.org/cases/1972/71-6698
Case: https://api.oyez.org/cases/1972/71-839
Case: https://api.oyez.org/cases/1972/71-1315
Case: https://api.oyez.org/cases/1972/72-77
Case: https://api.oyez.org/cases/1972/72-634
Case: https://api.oyez.org/cases/1972/72-5521
Case: https://api.oyez.org/cases/1972/71-738
Case: https://api.oyez.org/cases/1972/71-1694
Case: https://api.oyez.org/cases/1972/71-900
Case: https://api.oyez.org/cases/1972/72-1164
Case: https://api.oyez.org/cases/1972/71-6742
C

Case: https://api.oyez.org/cases/1973/72-700
Case: https://api.oyez.org/cases/1973/73-1283
Case: https://api.oyez.org/cases/1973/73-38
Case: https://api.oyez.org/cases/1973/73-190
Case: https://api.oyez.org/cases/1973/72-1231
Case: https://api.oyez.org/cases/1973/72-1319
Case: https://api.oyez.org/cases/1973/72-955
Case: https://api.oyez.org/cases/1973/73-718
Case: https://api.oyez.org/cases/1973/72-851
Case: https://api.oyez.org/cases/1973/72-1410
Case: https://api.oyez.org/cases/1973/72-1490
Case: https://api.oyez.org/cases/1973/73-434
Case: https://api.oyez.org/cases/1973/72-6449
Case: https://api.oyez.org/cases/1973/72-1517
Case: https://api.oyez.org/cases/1973/71-1669
Case: https://api.oyez.org/cases/1973/72-6902
Case: https://api.oyez.org/cases/1973/73-482
Case: https://api.oyez.org/cases/1973/72-1713
Case: https://api.oyez.org/cases/1973/72-1154
Case: https://api.oyez.org/cases/1973/72-1019
Case: https://api.oyez.org/cases/1973/73-568
Case: https://api.oyez.org/cases/1973/72-135

Case: https://api.oyez.org/cases/1974/73-1231
Case: https://api.oyez.org/cases/1974/73-1995
Case: https://api.oyez.org/cases/1974/73-1424
Case: https://api.oyez.org/cases/1974/74-22
Case: https://api.oyez.org/cases/1974/74-364
Case: https://api.oyez.org/cases/1974/73-628
Case: https://api.oyez.org/cases/1974/73-689
Case: https://api.oyez.org/cases/1974/73-1363
Case: https://api.oyez.org/cases/1974/74-70
Case: https://api.oyez.org/cases/1974/73-1233
Case: https://api.oyez.org/cases/1974/73-1966
Case: https://api.oyez.org/cases/1974/73-1245
Case: https://api.oyez.org/cases/1974/73-1994
Case: https://api.oyez.org/cases/1974/74-114
Case: https://api.oyez.org/cases/1974/73-1380
Case: https://api.oyez.org/cases/1974/73-1908
Case: https://api.oyez.org/cases/1974/73-200
Case: https://api.oyez.org/cases/1974/74-80
Case: https://api.oyez.org/cases/1974/73-1513
Case: https://api.oyez.org/cases/1974/73-300
Case: https://api.oyez.org/cases/1974/73-1701
Case: https://api.oyez.org/cases/1974/73-2055


Case: https://api.oyez.org/cases/1975/74-1481
Case: https://api.oyez.org/cases/1975/74-966
Case: https://api.oyez.org/cases/1975/75-5027
Case: https://api.oyez.org/cases/1975/75-73
Case: https://api.oyez.org/cases/1975/74-54
Case: https://api.oyez.org/cases/1975/75-246
Case: https://api.oyez.org/cases/1975/75-250
Case: https://api.oyez.org/cases/1975/36_orig
Case: https://api.oyez.org/cases/1975/74-768
Case: https://api.oyez.org/cases/1975/75-292
Case: https://api.oyez.org/cases/1975/75-369
Case: https://api.oyez.org/cases/1975/74-5968
Case: https://api.oyez.org/cases/1975/74-489
Case: https://api.oyez.org/cases/1975/74-100
Case: https://api.oyez.org/cases/1975/74-1607
Case: https://api.oyez.org/cases/1975/74-1492
Case: https://api.oyez.org/cases/1975/74-1148
Case: https://api.oyez.org/cases/1975/74-1243
Case: https://api.oyez.org/cases/1975/75-510
Case: https://api.oyez.org/cases/1975/75-5706
Case: https://api.oyez.org/cases/1975/75-584
Case: https://api.oyez.org/cases/1975/74-521
Cas

...

Case: https://api.oyez.org/cases/1969/86
Case: https://api.oyez.org/cases/1969/87
Case: https://api.oyez.org/cases/1969/88
Case: https://api.oyez.org/cases/1969/896
Case: https://api.oyez.org/cases/1969/927
Case: https://api.oyez.org/cases/1970/1059
Case: https://api.oyez.org/cases/1970/1066
Case: https://api.oyez.org/cases/1970/107
Case: https://api.oyez.org/cases/1970/108
Case: https://api.oyez.org/cases/1970/109
Case: https://api.oyez.org/cases/1970/113
Case: https://api.oyez.org/cases/1970/117
Case: https://api.oyez.org/cases/1970/118
Case: https://api.oyez.org/cases/1970/120
Case: https://api.oyez.org/cases/1970/121
Case: https://api.oyez.org/cases/1970/123
Case: https://api.oyez.org/cases/1970/124
Case: https://api.oyez.org/cases/1970/125
Case: https://api.oyez.org/cases/1970/1267
Case: https://api.oyez.org/cases/1970/133
Case: https://api.oyez.org/cases/1970/135
Case: https://api.oyez.org/cases/1970/136
Case: https://api.oyez.org/cases/1970/143
Case: https://api.oyez.org/cases/1

Case: https://api.oyez.org/cases/1987/87-352
Case: https://api.oyez.org/cases/1987/87-339
Case: https://api.oyez.org/cases/1987/87-354
Case: https://api.oyez.org/cases/1987/87-367
Case: https://api.oyez.org/cases/1987/87-399
Case: https://api.oyez.org/cases/1987/87-498
Case: https://api.oyez.org/cases/1987/87-499
Case: https://api.oyez.org/cases/1987/86-986
Case: https://api.oyez.org/cases/1987/87-526
Case: https://api.oyez.org/cases/1987/87-5428
Case: https://api.oyez.org/cases/1987/87-5277
Case: https://api.oyez.org/cases/1987/87-5002
Case: https://api.oyez.org/cases/1987/87-5468
Case: https://api.oyez.org/cases/1987/87-573
Case: https://api.oyez.org/cases/1987/87-5565
Case: https://api.oyez.org/cases/1987/87-578
Case: https://api.oyez.org/cases/1987/87-6
Case: https://api.oyez.org/cases/1987/87-645
Case: https://api.oyez.org/cases/1987/87-626
Case: https://api.oyez.org/cases/1987/87-654
Case: https://api.oyez.org/cases/1987/87-920
Case: https://api.oyez.org/cases/1987/9_orig
Case: h

Case: https://api.oyez.org/cases/2016/15-5991
Case: https://api.oyez.org/cases/2016/15-866
Case: https://api.oyez.org/cases/2016/15-927
Case: https://api.oyez.org/cases/2015/15-833
Case: https://api.oyez.org/cases/2011/11-1053
Case: https://api.oyez.org/cases/2015/15-8366
Case: https://api.oyez.org/cases/2015/15-789
Case: https://api.oyez.org/cases/2016/15-513
Case: https://api.oyez.org/cases/2016/15-797
Case: https://api.oyez.org/cases/2016/15-680
Case: https://api.oyez.org/cases/2016/15-8049
Case: https://api.oyez.org/cases/2016/15-1251
Case: https://api.oyez.org/cases/2017/15-1204
Case: https://api.oyez.org/cases/2016/15-1262
Case: https://api.oyez.org/cases/2016/14-1538
Case: https://api.oyez.org/cases/2016/15-8544
Case: https://api.oyez.org/cases/2016/14-1055
Case: https://api.oyez.org/cases/2016/15-423
Case: https://api.oyez.org/cases/2016/15-497
Case: https://api.oyez.org/cases/2016/15-649
Case: https://api.oyez.org/cases/2016/15-1111
Case: https://api.oyez.org/cases/2016/15-119

Case: https://api.oyez.org/cases/2018/17-9572
Case: https://api.oyez.org/cases/2018/18-15
Case: https://api.oyez.org/cases/2018/17-1484
Case: https://api.oyez.org/cases/2018/18-56
Case: https://api.oyez.org/cases/2018/18-302
Case: https://api.oyez.org/cases/2018/18-459
Case: https://api.oyez.org/cases/2018/18-489
Case: https://api.oyez.org/cases/2018/18-431
Case: https://api.oyez.org/cases/2018/18-315
Case: https://api.oyez.org/cases/2018/17-778
Case: https://api.oyez.org/cases/2018/17-9560
Case: https://api.oyez.org/cases/2018/18-389
Case: https://api.oyez.org/cases/2018/18-457
Case: https://api.oyez.org/cases/2018/18-481
Case: https://api.oyez.org/cases/2018/18-485
Case: https://api.oyez.org/cases/2018/18-525
Case: https://api.oyez.org/cases/2018/17-1705
Case: https://api.oyez.org/cases/2019/18-280
Case: https://api.oyez.org/cases/2018/18-266
Case: https://api.oyez.org/cases/2018/18-6210
Case: https://api.oyez.org/cases/2018/18-281
Case: https://api.oyez.org/cases/2018/18-422
Case: h

KeyError: 'decisions'

In [6]:
for f in cf.as_completed(futures_decisions):
    resp = f.result()
    href = resp.url
    case_id = futures_decisions_lookup[href]
    data[case_id]['decisions'].append(resp.json())
    print(f'Decision: {href}')

Decision: https://api.oyez.org/case_decision/case_decision/10746
Decision: https://api.oyez.org/case_decision/case_decision/10387
Decision: https://api.oyez.org/case_decision/case_decision/9272
Decision: https://api.oyez.org/case_decision/case_decision/8942
Decision: https://api.oyez.org/case_decision/case_decision/9859
Decision: https://api.oyez.org/case_decision/case_decision/10607
Decision: https://api.oyez.org/case_decision/case_decision/12863
Decision: https://api.oyez.org/case_decision/case_decision/11890
Decision: https://api.oyez.org/case_decision/case_decision/12177
Decision: https://api.oyez.org/case_decision/case_decision/11159
Decision: https://api.oyez.org/case_decision/case_decision/10367
Decision: https://api.oyez.org/case_decision/case_decision/13302
Decision: https://api.oyez.org/case_decision/case_decision/12413
Decision: https://api.oyez.org/case_decision/case_decision/10443
Decision: https://api.oyez.org/case_decision/case_decision/9445
Decision: https://api.oyez.or

Decision: https://api.oyez.org/case_decision/case_decision/9594
Decision: https://api.oyez.org/case_decision/case_decision/11262
Decision: https://api.oyez.org/case_decision/case_decision/16489
Decision: https://api.oyez.org/case_decision/case_decision/8260
Decision: https://api.oyez.org/case_decision/case_decision/13129
Decision: https://api.oyez.org/case_decision/case_decision/11731
Decision: https://api.oyez.org/case_decision/case_decision/9262
Decision: https://api.oyez.org/case_decision/case_decision/10809
Decision: https://api.oyez.org/case_decision/case_decision/13943
Decision: https://api.oyez.org/case_decision/case_decision/9139
Decision: https://api.oyez.org/case_decision/case_decision/10061
Decision: https://api.oyez.org/case_decision/case_decision/9140
Decision: https://api.oyez.org/case_decision/case_decision/13944
Decision: https://api.oyez.org/case_decision/case_decision/10942
Decision: https://api.oyez.org/case_decision/case_decision/12115
Decision: https://api.oyez.org

Decision: https://api.oyez.org/case_decision/case_decision/9037
Decision: https://api.oyez.org/case_decision/case_decision/13269
Decision: https://api.oyez.org/case_decision/case_decision/9510
Decision: https://api.oyez.org/case_decision/case_decision/11881
Decision: https://api.oyez.org/case_decision/case_decision/9489
Decision: https://api.oyez.org/case_decision/case_decision/11833
Decision: https://api.oyez.org/case_decision/case_decision/10426
Decision: https://api.oyez.org/case_decision/case_decision/13525
Decision: https://api.oyez.org/case_decision/case_decision/10678
Decision: https://api.oyez.org/case_decision/case_decision/11190
Decision: https://api.oyez.org/case_decision/case_decision/10952
Decision: https://api.oyez.org/case_decision/case_decision/13140
Decision: https://api.oyez.org/case_decision/case_decision/13438
Decision: https://api.oyez.org/case_decision/case_decision/11862
Decision: https://api.oyez.org/case_decision/case_decision/12962
Decision: https://api.oyez.o

Decision: https://api.oyez.org/case_decision/case_decision/13132
Decision: https://api.oyez.org/case_decision/case_decision/8673
Decision: https://api.oyez.org/case_decision/case_decision/13394
Decision: https://api.oyez.org/case_decision/case_decision/9620
Decision: https://api.oyez.org/case_decision/case_decision/9275
Decision: https://api.oyez.org/case_decision/case_decision/9194
Decision: https://api.oyez.org/case_decision/case_decision/9394
Decision: https://api.oyez.org/case_decision/case_decision/13346
Decision: https://api.oyez.org/case_decision/case_decision/12294
Decision: https://api.oyez.org/case_decision/case_decision/10764
Decision: https://api.oyez.org/case_decision/case_decision/10396
Decision: https://api.oyez.org/case_decision/case_decision/10123
Decision: https://api.oyez.org/case_decision/case_decision/9545
Decision: https://api.oyez.org/case_decision/case_decision/13294
Decision: https://api.oyez.org/case_decision/case_decision/12319
Decision: https://api.oyez.org/

Decision: https://api.oyez.org/case_decision/case_decision/9673
Decision: https://api.oyez.org/case_decision/case_decision/10784
Decision: https://api.oyez.org/case_decision/case_decision/10576
Decision: https://api.oyez.org/case_decision/case_decision/10402
Decision: https://api.oyez.org/case_decision/case_decision/13062
Decision: https://api.oyez.org/case_decision/case_decision/10945
Decision: https://api.oyez.org/case_decision/case_decision/10582
Decision: https://api.oyez.org/case_decision/case_decision/13358
Decision: https://api.oyez.org/case_decision/case_decision/13193
Decision: https://api.oyez.org/case_decision/case_decision/11966
Decision: https://api.oyez.org/case_decision/case_decision/8335
Decision: https://api.oyez.org/case_decision/case_decision/11603
Decision: https://api.oyez.org/case_decision/case_decision/9835
Decision: https://api.oyez.org/case_decision/case_decision/9882
Decision: https://api.oyez.org/case_decision/case_decision/12239
Decision: https://api.oyez.or

...

Decision: https://api.oyez.org/case_decision/case_decision/16600
Decision: https://api.oyez.org/case_decision/case_decision/16601
Decision: https://api.oyez.org/case_decision/case_decision/16592
Decision: https://api.oyez.org/case_decision/case_decision/16583
Decision: https://api.oyez.org/case_decision/case_decision/16621
Decision: https://api.oyez.org/case_decision/case_decision/16620
Decision: https://api.oyez.org/case_decision/case_decision/16585
Decision: https://api.oyez.org/case_decision/case_decision/16598
Decision: https://api.oyez.org/case_decision/case_decision/16599
Decision: https://api.oyez.org/case_decision/case_decision/16579
Decision: https://api.oyez.org/case_decision/case_decision/16616
Decision: https://api.oyez.org/case_decision/case_decision/16582
Decision: https://api.oyez.org/case_decision/case_decision/16578
Decision: https://api.oyez.org/case_decision/case_decision/16612
Decision: https://api.oyez.org/case_decision/case_decision/16577
Decision: https://api.oye

Decision: https://api.oyez.org/case_decision/case_decision/16509
Decision: https://api.oyez.org/case_decision/case_decision/16717
Decision: https://api.oyez.org/case_decision/case_decision/16536
Decision: https://api.oyez.org/case_decision/case_decision/16514
Decision: https://api.oyez.org/case_decision/case_decision/16515
Decision: https://api.oyez.org/case_decision/case_decision/16688
Decision: https://api.oyez.org/case_decision/case_decision/16444
Decision: https://api.oyez.org/case_decision/case_decision/16522
Decision: https://api.oyez.org/case_decision/case_decision/16664
Decision: https://api.oyez.org/case_decision/case_decision/16507
Decision: https://api.oyez.org/case_decision/case_decision/16508
Decision: https://api.oyez.org/case_decision/case_decision/16524
Decision: https://api.oyez.org/case_decision/case_decision/16387
Decision: https://api.oyez.org/case_decision/case_decision/16396
Decision: https://api.oyez.org/case_decision/case_decision/16381
Decision: https://api.oye

Decision: https://api.oyez.org/case_decision/case_decision/16517
Decision: https://api.oyez.org/case_decision/case_decision/16519
Decision: https://api.oyez.org/case_decision/case_decision/16768
Decision: https://api.oyez.org/case_decision/case_decision/16525
Decision: https://api.oyez.org/case_decision/case_decision/16526
Decision: https://api.oyez.org/case_decision/case_decision/16767
Decision: https://api.oyez.org/case_decision/case_decision/16781
Decision: https://api.oyez.org/case_decision/case_decision/16793
Decision: https://api.oyez.org/case_decision/case_decision/16531
Decision: https://api.oyez.org/case_decision/case_decision/16772
Decision: https://api.oyez.org/case_decision/case_decision/16785
Decision: https://api.oyez.org/case_decision/case_decision/16783
Decision: https://api.oyez.org/case_decision/case_decision/16734
Decision: https://api.oyez.org/case_decision/case_decision/16701
Decision: https://api.oyez.org/case_decision/case_decision/16704
Decision: https://api.oye

Decision: https://api.oyez.org/case_decision/case_decision/16895
Decision: https://api.oyez.org/case_decision/case_decision/16842
Decision: https://api.oyez.org/case_decision/case_decision/16911
Decision: https://api.oyez.org/case_decision/case_decision/16883
Decision: https://api.oyez.org/case_decision/case_decision/16880
Decision: https://api.oyez.org/case_decision/case_decision/16908
Decision: https://api.oyez.org/case_decision/case_decision/16899
Decision: https://api.oyez.org/case_decision/case_decision/16890
Decision: https://api.oyez.org/case_decision/case_decision/16901
Decision: https://api.oyez.org/case_decision/case_decision/16900
Decision: https://api.oyez.org/case_decision/case_decision/16897
Decision: https://api.oyez.org/case_decision/case_decision/16902
Decision: https://api.oyez.org/case_decision/case_decision/16838
Decision: https://api.oyez.org/case_decision/case_decision/16913
Decision: https://api.oyez.org/case_decision/case_decision/16847
Decision: https://api.oye

Decision: https://api.oyez.org/case_decision/case_decision/17020
Decision: https://api.oyez.org/case_decision/case_decision/17022
Decision: https://api.oyez.org/case_decision/case_decision/17009
Decision: https://api.oyez.org/case_decision/case_decision/17027
Decision: https://api.oyez.org/case_decision/case_decision/17033
Decision: https://api.oyez.org/case_decision/case_decision/17034
Decision: https://api.oyez.org/case_decision/case_decision/17042
Decision: https://api.oyez.org/case_decision/case_decision/17041
Decision: https://api.oyez.org/case_decision/case_decision/17038
Decision: https://api.oyez.org/case_decision/case_decision/17040
Decision: https://api.oyez.org/case_decision/case_decision/17024
Decision: https://api.oyez.org/case_decision/case_decision/17043
Decision: https://api.oyez.org/case_decision/case_decision/17035
Decision: https://api.oyez.org/case_decision/case_decision/17057
Decision: https://api.oyez.org/case_decision/case_decision/17037
Decision: https://api.oye

In [9]:
for f in cf.as_completed(futures_opinions):
    resp = f.result()
    href = resp.url
    case_id = futures_opinions_lookup[href]
    data[case_id]['opinions'].append(resp.json())
    print(f'Opinion: {href}')

Opinion: https://api.oyez.org/case_document/written_opinion/11769
Opinion: https://api.oyez.org/case_document/written_opinion/11989
Opinion: https://api.oyez.org/case_document/written_opinion/5570
Opinion: https://api.oyez.org/case_document/written_opinion/2683
Opinion: https://api.oyez.org/case_document/written_opinion/13464
Opinion: https://api.oyez.org/case_document/written_opinion/4034
Opinion: https://api.oyez.org/case_document/written_opinion/6003
Opinion: https://api.oyez.org/case_document/written_opinion/14206
Opinion: https://api.oyez.org/case_document/written_opinion/118
Opinion: https://api.oyez.org/case_document/written_opinion/9031
Opinion: https://api.oyez.org/case_document/written_opinion/12844
Opinion: https://api.oyez.org/case_document/written_opinion/1314
Opinion: https://api.oyez.org/case_document/written_opinion/7841
Opinion: https://api.oyez.org/case_document/written_opinion/13368
Opinion: https://api.oyez.org/case_document/written_opinion/13097
Opinion: https://ap

Opinion: https://api.oyez.org/case_document/written_opinion/5592
Opinion: https://api.oyez.org/case_document/written_opinion/13222
Opinion: https://api.oyez.org/case_document/written_opinion/6654
Opinion: https://api.oyez.org/case_document/written_opinion/8526
Opinion: https://api.oyez.org/case_document/written_opinion/6353
Opinion: https://api.oyez.org/case_document/written_opinion/9329
Opinion: https://api.oyez.org/case_document/written_opinion/10300
Opinion: https://api.oyez.org/case_document/written_opinion/12884
Opinion: https://api.oyez.org/case_document/written_opinion/12056
Opinion: https://api.oyez.org/case_document/written_opinion/6074
Opinion: https://api.oyez.org/case_document/written_opinion/444
Opinion: https://api.oyez.org/case_document/written_opinion/3803
Opinion: https://api.oyez.org/case_document/written_opinion/6075
Opinion: https://api.oyez.org/case_document/written_opinion/16471
Opinion: https://api.oyez.org/case_document/written_opinion/13435
Opinion: https://api

Opinion: https://api.oyez.org/case_document/written_opinion/12146
Opinion: https://api.oyez.org/case_document/written_opinion/12555
Opinion: https://api.oyez.org/case_document/written_opinion/3526
Opinion: https://api.oyez.org/case_document/written_opinion/13676
Opinion: https://api.oyez.org/case_document/written_opinion/1103
Opinion: https://api.oyez.org/case_document/written_opinion/13990
Opinion: https://api.oyez.org/case_document/written_opinion/12556
Opinion: https://api.oyez.org/case_document/written_opinion/13989
Opinion: https://api.oyez.org/case_document/written_opinion/15615
Opinion: https://api.oyez.org/case_document/written_opinion/3491
Opinion: https://api.oyez.org/case_document/written_opinion/14621
Opinion: https://api.oyez.org/case_document/written_opinion/9694
Opinion: https://api.oyez.org/case_document/written_opinion/2777
Opinion: https://api.oyez.org/case_document/written_opinion/15622
Opinion: https://api.oyez.org/case_document/written_opinion/12026
Opinion: https:

Opinion: https://api.oyez.org/case_document/written_opinion/16247
Opinion: https://api.oyez.org/case_document/written_opinion/7992
Opinion: https://api.oyez.org/case_document/written_opinion/8376
Opinion: https://api.oyez.org/case_document/written_opinion/5174
Opinion: https://api.oyez.org/case_document/written_opinion/8937
Opinion: https://api.oyez.org/case_document/written_opinion/14659
Opinion: https://api.oyez.org/case_document/written_opinion/12426
Opinion: https://api.oyez.org/case_document/written_opinion/11588
Opinion: https://api.oyez.org/case_document/written_opinion/2162
Opinion: https://api.oyez.org/case_document/written_opinion/3912
Opinion: https://api.oyez.org/case_document/written_opinion/7991
Opinion: https://api.oyez.org/case_document/written_opinion/10234
Opinion: https://api.oyez.org/case_document/written_opinion/4753
Opinion: https://api.oyez.org/case_document/written_opinion/1881
Opinion: https://api.oyez.org/case_document/written_opinion/16178
Opinion: https://ap

Opinion: https://api.oyez.org/case_document/written_opinion/10085
Opinion: https://api.oyez.org/case_document/written_opinion/16320
Opinion: https://api.oyez.org/case_document/written_opinion/13393
Opinion: https://api.oyez.org/case_document/written_opinion/5955
Opinion: https://api.oyez.org/case_document/written_opinion/13394
Opinion: https://api.oyez.org/case_document/written_opinion/13381
Opinion: https://api.oyez.org/case_document/written_opinion/14665
Opinion: https://api.oyez.org/case_document/written_opinion/15376
Opinion: https://api.oyez.org/case_document/written_opinion/1319
Opinion: https://api.oyez.org/case_document/written_opinion/3085
Opinion: https://api.oyez.org/case_document/written_opinion/15267
Opinion: https://api.oyez.org/case_document/written_opinion/15413
Opinion: https://api.oyez.org/case_document/written_opinion/1937
Opinion: https://api.oyez.org/case_document/written_opinion/10689
Opinion: https://api.oyez.org/case_document/written_opinion/1865
Opinion: https:

Opinion: https://api.oyez.org/case_document/written_opinion/8099
Opinion: https://api.oyez.org/case_document/written_opinion/8381
Opinion: https://api.oyez.org/case_document/written_opinion/7109
Opinion: https://api.oyez.org/case_document/written_opinion/1438
Opinion: https://api.oyez.org/case_document/written_opinion/1754
Opinion: https://api.oyez.org/case_document/written_opinion/14982
Opinion: https://api.oyez.org/case_document/written_opinion/2306
Opinion: https://api.oyez.org/case_document/written_opinion/1436
Opinion: https://api.oyez.org/case_document/written_opinion/1437
Opinion: https://api.oyez.org/case_document/written_opinion/1751
Opinion: https://api.oyez.org/case_document/written_opinion/1407
Opinion: https://api.oyez.org/case_document/written_opinion/5490
Opinion: https://api.oyez.org/case_document/written_opinion/13074
Opinion: https://api.oyez.org/case_document/written_opinion/14707
Opinion: https://api.oyez.org/case_document/written_opinion/6749
Opinion: https://api.o

Opinion: https://api.oyez.org/case_document/written_opinion/9507
Opinion: https://api.oyez.org/case_document/written_opinion/4300
Opinion: https://api.oyez.org/case_document/written_opinion/7329
Opinion: https://api.oyez.org/case_document/written_opinion/4939
Opinion: https://api.oyez.org/case_document/written_opinion/8592
Opinion: https://api.oyez.org/case_document/written_opinion/4301
Opinion: https://api.oyez.org/case_document/written_opinion/9351
Opinion: https://api.oyez.org/case_document/written_opinion/7390
Opinion: https://api.oyez.org/case_document/written_opinion/1604
Opinion: https://api.oyez.org/case_document/written_opinion/3053
Opinion: https://api.oyez.org/case_document/written_opinion/6504
Opinion: https://api.oyez.org/case_document/written_opinion/14623
Opinion: https://api.oyez.org/case_document/written_opinion/301
Opinion: https://api.oyez.org/case_document/written_opinion/958
Opinion: https://api.oyez.org/case_document/written_opinion/6505
Opinion: https://api.oyez.

Opinion: https://api.oyez.org/case_document/written_opinion/5995
Opinion: https://api.oyez.org/case_document/written_opinion/12954
Opinion: https://api.oyez.org/case_document/written_opinion/1225
Opinion: https://api.oyez.org/case_document/written_opinion/13584
Opinion: https://api.oyez.org/case_document/written_opinion/13876
Opinion: https://api.oyez.org/case_document/written_opinion/4857
Opinion: https://api.oyez.org/case_document/written_opinion/6614
Opinion: https://api.oyez.org/case_document/written_opinion/13810
Opinion: https://api.oyez.org/case_document/written_opinion/2038
Opinion: https://api.oyez.org/case_document/written_opinion/2847
Opinion: https://api.oyez.org/case_document/written_opinion/7493
Opinion: https://api.oyez.org/case_document/written_opinion/3158
Opinion: https://api.oyez.org/case_document/written_opinion/4419
Opinion: https://api.oyez.org/case_document/written_opinion/8735
Opinion: https://api.oyez.org/case_document/written_opinion/9740
Opinion: https://api.

In [10]:
print(f'Successfully downloaded {len(data)} cases')

Successfully downloaded 8209 cases


## Export the data to JSON files
oyez.json will be the compact minified version, whereas oyez_pretty.json will be the indented and human-readable version.
Expect them to have sizes of 220MB and 430MB respectively.

In [11]:
# Only export the values of the dictionary (which are the cases in JSON)
values = list(data.values())

with open('oyez.json', 'w') as f:
    json.dump(values, f)
    
with open('oyez_pretty.json', 'w') as f:
    json.dump(values, f, indent=4)